**CONCLUSION**: Calculating "wasted time" in bed (minutes in bed - minutes asleep) provided insights into sleep quality. Notably, most users showed low "wasted time," suggesting efficient sleep cycles and potentially good sleep quality.

Next we see the Relationship between avg minutes of sleep and calories burned per user

SELECT ACTIVITY.Id, 
  ROUND(avg(ACTIVITY.total_steps),2) AS AvgTotalSteps,
   ROUND(avg(ACTIVITY.calories),2) AS AvgCalories,
   ROUND(avg(SLEEP.total_minutes_asleep),2) AS AvgTotalMinutesAsleep
FROM `fluid-tuner-406721.FitBit_Case_Study.daily_activity` AS ACTIVITY
  INNER JOIN fluid-tuner-406721.FitBit_Case_Study.daily_sleep AS SLEEP
  ON ACTIVITY.id = SLEEP.id
GROUP BY ACTIVITY.Id

**CONCLUSION**: The relationship between People who slept better and burned calories seems to be positive!

### 5.10  METs analysis  <a class="anchor" id="METs"></a>

* Q. What is METs?
     * MET: The ratio of the work metabolic rate to the resting metabolic rate. One MET is defined as 1 kcal/kg/hour. The **metabolic equivalent of task (MET)** is the objective measure of the ratio of the rate at which a person expends energy equivalent to the energy cost of sitting down.

* MET values here are divided by 10 because according to meta data of the dataset: "All MET values exported from Fitabase are multiplied by 10. Please divide by 10 to get accurate MET values"

--METs

SELECT DISTINCT temp1.id, 
    temp1.activity_minute, 
    SUM(temp1.METs)/24 as avg_mets, 
    temp2.Calories
FROM `fluid-tuner-406721.FitBit_Case_Study.METs_minute` as temp1
INNER JOIN fluid-tuner-406721.FitBit_Case_Study.daily_activity AS temp2
ON temp1.id = temp2.id 
Group By temp1.Id, temp1.activity_minute, temp2.Calories
Order by Id,activity_minute;

![](http://)

**CONCLUSION**: Our data reveals a positive link between METs (activity level) and calorie burn.

## <font color='#E2A76F'>6. ACT Phase</font> <a class="anchor" id="act"></a>

### 6.1 Conclusion <a class="anchor" id="conclusion"></a> 

The global fitness tracker market size was valued at 47 billion dollar in 2022 & is projected to grow from  53.94 billion dollar  in 2023 to $182.90 billion by 2030. To succeed in this competitive market, the fitness band company needs to have a strong marketing strategy. By diving into Fitbit user data, we gain invaluable insights into real-world fitness watch usage. This unveils both market gaps and user preferences, empowering us to tailor Bellabeat and highlight features that truly resonate with the audience. The most important data points is as follows:

* 1. _High engagement_: 64% of users logged data for the entire data time period of 31 days. This shows that users are actively tracking their progress.

* 2. _Potential for improvement_: 8% of non-usage (72 missed wearings) indicates room for improvement in encouraging consistent wear. Tuesday emerges as the day with the most missed wearings. 

* 3. _CDC Activity Length Recommendations_:  On average, 27 of the 33 users met the CDC Activity Length Recommendations, while 6 did not (Very Active, Fairly Active & Lightly active minutes together). 

* 4. _High average sedentary minutes_: Average sedentary minutes of 931.78 minutes (15 hours 31.78 minutes). The average sedentary time might be higher than recommended guidelines. For instance, some health organizations recommend limiting sitting time to less than 8 hours per day. 81% of the time by all users was spent on sedentary time hence they possibly have a less active lifestyle.
 
* 5.  Very active minutes, fairly active minutes & Lightly active minutes rise a little during Saturdays and similarly sedendary activity does fall a little suggesting users are more likely to engage in physical activities on weekends compared to weekdays. Weekends present an opportunity to encourage and sustain healthy activity levels. By delving deeper into the reasons behind this trend and tailoring interventions accordingly, you can help users maintain a more active lifestyle throughout the week.
 
* 6. 5 PM -7 PM is the duration of highest calories burned and hence could be the desired time of workout for all the users combined on an average.
  
* 7. The relationship between People who slept better and burned calories seems to be positive. Users averaging around 7 hours of sleep consistently. 

### 6.2 Recommendations <a class="anchor" id="recommendations"></a>

**_1. Digital Marketting_**:

   **A. Partner with influencers to promote Ivy health tracker**: As seen in the above analysis, all 33 users have logged in  step, activity instensity and calorie burned features on Fitbit, Bellabeat can effectively reach its target audience by partnering with relevant influencers: a fitness expert and a registered dietitian. They can also offer exclusive discounts and promotions to influencers' followers. Schedule social media posts and email campaigns for early morning audiences.
   
   
   **B. Run targeted advertising campaigns on social media and search engines**: The data tells one thing: That 81% of the time by all users was spent on sedentary time hence they possibly have a desk job. Therefore we can Craft targeted ads showcasing Bellabeat's features like activity reminders and step challenges as the antidote to sedentary desk jobs. Target platforms frequented by professional women on Facebook, LinkedIn, and fitness-focused Instagram accounts.
   
  **C.Create engaging social media content**: Since social media is the prefered mode of advertisment for Bellabeat. We can feature real Bellabeat users sharing their fitness journeys, showcasing product features, and offering helpful tips. Feature inspiring stories of women overcoming challenges and achieving their goals with Bellabeat by their side.
   
   **D. Search Engine Optimization (SEO)** : Optimize BellaBeat's website and content for relevant keywords like "women's fitness tracker," "desk job wellness," and " menstrual cycle recorder", "pregnancy-safe workouts" to reach potential customers organically.
   
 **_2. Embrace Heart and Weight_** : Analyze the underutilized features like heart rate and weight tracking. Consider developing content, challenges, or integrations focused on these aspects to cater to a wider range of user interests and goals which could be a possible market gap. 

**_3.Segmentation and Targeting_**:

   **A. Data insights reveal a key opportunity: some customers bought the fitbit band but haven't fully embraced its potential. For such existing customers following approach can be followed**

* 1. _Personalized nudges_: Trigger targeted emails highlighting the benefits of increased activity based on individual user data. Showcase how small, attainable steps lead to big health improvements.
* 2. _Playful motivators_: Send gentle app notifications reminding users to move, offering playful challenges based on their inactivity patterns. Celebrate milestones with virtual high-fives and encourage progress with fun rewards.
* 3. _Sleep for Strength_: Leverage sleep data to send personalized app notifications with sleep coaching tips. Emphasize the link between quality sleep and boosted energy, improved activity levels, and overall well-being.
* 4. _Challenge them to shine_: Design bite-sized nightly sleep challenges, like achieving a specific sleep duration or maintaining a consistent sleep schedule, to encourage healthy sleep habits in a fun and engaging way.

**B. Position the band as the "after-work empowerment tool"**: Since Post-office workouts seem to be popular amongst our sample group we can frame evening workouts as a way to de-stress, boost energy, and reclaim control after a busy day. 
      
**_4. Emphasize data and insights_** : Fitbit is known for its data-driven approach to fitness tracking. The brand uses data to provide users with insights into their activity levels, sleep patterns, and overall health. Bellabeat can differentiate itself by offering even more advanced data and insights, such as personalized fitness plans, health recommendations, and integration with other health apps.

**_5. Community Connection_** : Foster a vibrant online community through social media groups, interactive forums, and virtual workout sessions. Encourage users to share tips, support each other, and celebrate achievements together.

**_6. Product Development and Improvement_**:
  * A. Our data shows the sample users as early risers. Develop features specifically for early mornings, like customizable sunrise alarms, guided meditation sessions for stress reduction, or benefit of a quick workout routines for before work.
  * B. Weekly progress report with segregation of weekday-weekend reports can be given to users to keep them motivated for the whole week. 

**Additional Considerations**
* 1. Data privacy: Ensure you comply with all data privacy regulations and protect user data.
* 2. Ethical marketing: Avoid using misleading or deceptive marketing practices.
* 3. Transparency: Be transparent about your data collection and usage practices.